In [5]:
import sqlite3
import time


def solve_mTSP_greedy(instance_id, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Instance information
    cursor.execute("SELECT nr_cities, nr_salesmen FROM instances WHERE instance_id = ?", (instance_id,))
    instance = cursor.fetchone()
    if not instance:
        print(f"Instance {instance_id} not found.")
        conn.close()
        return

    nr_cities, nr_salesmen = instance

    # City coordinates
    cursor.execute("SELECT city_id, x, y FROM cities WHERE instance_id = ?", (instance_id,))
    cities = cursor.fetchall()

    # Create the distance matrix

    distance_matrix = [[
        ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5 for _, x2, y2 in cities
    ] for _, x1, y1 in cities]

    # Solve with Greedy algorithm
    start_time = time.time()

    # Initialize routes for each salesman
    routes = [[] for _ in range(nr_salesmen)]
    visited = set()  
    visited.add(0)  # Depot is always visited

    # Assign at least one city to each salesman
    unvisited_cities = set(range(1, nr_cities + 1))  
    for salesman_id in range(nr_salesmen):
        if unvisited_cities:
            # Find the nearest city to the depot for each salesman
            nearest_city = min(unvisited_cities, key=lambda city: distance_matrix[0][city])
            routes[salesman_id].append(0) 
            routes[salesman_id].append(nearest_city)
            visited.add(nearest_city)
            unvisited_cities.remove(nearest_city)

    # Assign remaining cities to the nearest occupied city
    while unvisited_cities:
        nearest_city = None
        nearest_distance = float('inf')
        nearest_salesman = None

        # Find the nearest unvisited city to any occupied city
        for salesman_id, route in enumerate(routes):
            if not route:
                continue
            current_city = route[-1]  # Last city in the current salesman's route
            for city_id in unvisited_cities:
                distance = distance_matrix[current_city][city_id]
                if distance < nearest_distance:
                    nearest_city = city_id
                    nearest_distance = distance
                    nearest_salesman = salesman_id

        # Assign the nearest city to the corresponding salesman
        if nearest_city is not None and nearest_salesman is not None:
            routes[nearest_salesman].append(nearest_city)
            visited.add(nearest_city)
            unvisited_cities.remove(nearest_city)

    # All salesmen should return to the depot
    for route in routes:
        if route and route[-1] != 0:
            route.append(0)

    end_time = time.time()
    time_taken = end_time - start_time

    # Calculate the total cost
    total_cost = 0
    distances = []
    for route in routes:
        route_distance = sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        distances.append(route_distance)
        total_cost += route_distance

    # Calculate distance gap 
    distance_gap  = max(distances) - min(distances)

    # Calculate efficiency 
    epsilon = 1e-6  # offset for division by zero
    efficiency = total_cost / (time_taken + epsilon)


    # Insert results into the algorithms table
    cursor.execute("""
        INSERT OR REPLACE INTO algorithms (instance_id, strategy, total_cost, time_taken, distance_gap, efficiency)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (instance_id, "Greedy", total_cost, time_taken, distance_gap, efficiency))

    # Clear previous routes and insert routes into the routes table
    cursor.execute("DELETE FROM routes WHERE instance_id = ? AND strategy = ?", (instance_id, "Greedy"))

    for salesman_id, route in enumerate(routes):
        cursor.execute("""
            INSERT INTO routes (instance_id, strategy, salesman_id, route)
            VALUES (?, ?, ?, ?)
        """, (instance_id, "Greedy", salesman_id, str(route)))

    conn.commit()
    conn.close()

    print(f"Instance {instance_id} solved using Greedy.")

def solve_mTSP_greedy_for_all_instances(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Select instances that have not been solved with Greedy
    cursor.execute("""
        SELECT instance_id 
        FROM instances 
        WHERE instance_id NOT IN (
            SELECT instance_id 
            FROM algorithms 
            WHERE strategy = 'Greedy'
        )
    """)
    instances = cursor.fetchall()

    if not instances:
        print("No unsolved instances found for Greedy.")
        conn.close()
        return

    for (instance_id,) in instances:
        solve_mTSP_greedy(instance_id, db_file)

    conn.close()


solve_mTSP_greedy_for_all_instances(db_file="../train_mTSP.sqlite3")

Instance 138 solved using Greedy.
Instance 139 solved using Greedy.
Instance 140 solved using Greedy.
